In [23]:
from collections import UserDict

In [12]:
class MyMeta(type):
    def __new__(mcls, name, bases, cls_dict, **kwargs):
        print("mcls", mcls, type(mcls))
        print("name", name, type(name))
        print("bases", bases, type(bases))
        print("cls_dict", cls_dict, type(cls_dict))
        print("kwargs", kwargs, type(kwargs))
        return super().__new__(mcls, name, bases, cls_dict,)

In [13]:
class MyClass(metaclass = MyMeta):
    pass

mcls <class '__main__.MyMeta'> <class 'type'>
name MyClass <class 'str'>
bases () <class 'tuple'>
cls_dict {'__module__': '__main__', '__qualname__': 'MyClass'} <class 'dict'>
kwargs {} <class 'dict'>


where and how is that cls_dict get created?

the is method `__prepare__` for this. the default implementation returns empty dict. we can override it

In [14]:
class MyMeta(type):
    def __prepare__(name, bases, **kwargs): #static method
        print("MyMeta,__prepare__ was called..")
        print("name", name, type(name))
        print("kwargs", kwargs, type(kwargs))
        return {"a":100, "b":200}
    
    def __new__(mcls, name, bases, cls_dict, **kwargs): #static method
        print("MyMeta.__new__ was called..")
        print("mcls", mcls, type(mcls))
        print("name", name, type(name))
        print("bases", bases, type(bases))
        print("cls_dict", cls_dict, type(cls_dict))
        print("kwargs", kwargs, type(kwargs))
        return super().__new__(mcls, name, bases, cls_dict, )

In [15]:
class MyClass(metaclass = MyMeta, kw = 1, kw2 = 2):
    pass

MyMeta,__prepare__ was called..
name MyClass <class 'str'>
kwargs {'kw': 1, 'kw2': 2} <class 'dict'>
MyMeta.__new__ was called..
mcls <class '__main__.MyMeta'> <class 'type'>
name MyClass <class 'str'>
bases () <class 'tuple'>
cls_dict {'a': 100, 'b': 200, '__module__': '__main__', '__qualname__': 'MyClass'} <class 'dict'>
kwargs {'kw': 1, 'kw2': 2} <class 'dict'>


before the new method is called, python calls the prepare method to get base dictionary to use for the class dictionary. than it injects whatever it needs to and pass it to the new method

In [17]:
type.__prepare__()

{}

In [18]:
class MyMeta(type):
      
    def __new__(mcls, name, bases, cls_dict, **kwargs): 
        cls_dict.update(kwargs)
        return super().__new__(mcls, name, bases, cls_dict, )

In [19]:
class MyClass(metaclass = MyMeta, kw = 1, kw2 = 2):
    pass

In [20]:
vars(MyClass)

mappingproxy({'__module__': '__main__',
              'kw': 1,
              'kw2': 2,
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

In [21]:
class MyMeta(type):
    def __prepare__(name, bases, **kwargs): 
        return kwargs
    
    def __new__(mcls, name, bases, cls_dict): # we do need to implement new, if we do __prepare__ and have kwargs
        return super().__new__(mcls, name, bases, cls_dict, )

`__prepare__` needs to return an object, which is an instance of dict, but it doesn't have to be a dictionary (OrderedDict, for example). UserDict won't do:

In [27]:
class CustomDict(UserDict):
    def __setitem__(self, key, value):
        print(f"Setting {key} = {value} in custom dictionary")
        super().__setitem__(key, value)
    
    def __getitem__(self, key):
        print(f"Getting {key} from custom dictionary")
        return int(super().__getitem__(key))

In [28]:
 class MyMeta(type):
    def __prepare__(name, bases):
        return CustomDict()

In [29]:
class MyClass(metaclass = MyMeta, ):
    pass

Getting __name__ from custom dictionary
Setting __module__ = __main__ in custom dictionary
Setting __qualname__ = MyClass in custom dictionary


TypeError: type.__new__() argument 3 must be dict, not CustomDict

In [30]:
issubclass(UserDict, dict)

False

In [31]:
class CustomDict(dict):
    def __setitem__(self, key, value):
        print(f"Setting {key} = {value} in custom dictionary")
        super().__setitem__(key, value)
    
    def __getitem__(self, key):
        print(f"Getting {key} from custom dictionary")
        return int(super().__getitem__(key))

In [32]:
 class MyMeta(type):
    def __prepare__(name, bases):
        return CustomDict()

In [33]:
class MyClass(metaclass = MyMeta, ):
    pass

Getting __name__ from custom dictionary
Setting __module__ = __main__ in custom dictionary
Setting __qualname__ = MyClass in custom dictionary
